In [1]:
import pandas as pd
import numpy as np
import plotnine 
from plotnine import *
import os, sys, gc
#from tqdm.notebook import tqdm

C:\Users\Minji\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\Minji\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
C:\Users\Minji\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64

In [3]:
path = '../data/books/'

In [ ]:
books = pd.read_csv(path + "books.csv")
book_tags = pd.read_csv(path + "book_tags.csv")
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
tags = pd.read_csv(path + "tags.csv")
to_read = pd.read_csv(path + "to_read.csv")

In [ ]:
train['book_id'] = train['book_id'].astype(str)
test['book_id'] = test['book_id'].astype(str)
books['book_id'] = books['book_id'].astype(str)

In [ ]:
sol = test.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
gt = {}
for user in tqdm(sol['user_id'].unique()): 
    gt[user] = list(sol[sol['user_id'] == user]['unique'].values[0])

In [ ]:
# 한사람당 100권의 책 추천

rec_df = pd.DataFrame()
rec_df['user_id'] = train['user_id'].unique()

## Baseline
> 통계 기반 모델

In [ ]:
books.sort_values(by='books_count', ascending=False)[0:5]

In [ ]:
popular_rec_model = books.sort_values(by='books_count', ascending=False)['book_id'].values[0:500]

In [ ]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    for rec in popular_rec_model[0:200]: 
        rec_list.append(rec)
    total_rec_list[user] = rec_list

In [ ]:
import six
import math

# https://github.com/kakao-arena/brunch-article-recommendation/blob/master/evaluate.py

class evaluate():
    def __init__(self, recs, gt, topn=100):
        self.recs = recs
        self.gt = gt 
        self.topn = topn 
        
    def _ndcg(self):
        Q, S = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            dcg = 0.0
            idcg = sum([1.0 / math.log(i + 2, 2) for i in range(min(len(seen), len(rec)))])
            for i, r in enumerate(rec):
                if r not in seen:
                    continue
                rank = i + 1
                dcg += 1.0 / math.log(rank + 1, 2)
            ndcg = dcg / idcg
            S += ndcg
            Q += 1
        return S / Q


    def _map(self):
        n, ap = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            _ap, correct = 0.0, 0.0
            for i, r in enumerate(rec):
                if r in seen:
                    correct += 1
                    _ap += (correct / (i + 1.0))
            _ap /= min(len(seen), len(rec))
            ap += _ap
            n += 1.0
        return ap / n


    def _entropy_diversity(self):
        sz = float(len(self.recs)) * self.topn
        freq = {}
        for u, rec in six.iteritems(self.recs):
            for r in rec:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent
    
    def _evaluate(self):
        print('MAP@%s: %s' % (self.topn, self._map()))
        print('NDCG@%s: %s' % (self.topn, self._ndcg()))
        print('EntDiv@%s: %s' % (self.topn, self._entropy_diversity()))

In [ ]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

### Baseline 응용
- 많이 글은 글중에서도 평점이 높은 글들을 우선적으로 추천
- 내가 좋아하는 작가의 글을 우선적으로 추천
- 장바구니에 담긴 글과 작가의 글을 우선적으로 추천
- 읽은 글의 시리즈글이 나오면 추천 (해리포터 마법사의 돌 -> 비밀의 방)
- 최신의 글을 추천

In [ ]:
train = pd.merge(train, books[['book_id', 'authors', 'ratings_count']], how='left', on='book_id')

In [ ]:
agg = train.groupby(['user_id','authors'])['authors'].agg({'count'}).reset_index()
agg = agg.sort_values(by='count', ascending=False)
agg.head()

In [ ]:
author_books = books[['book_id', 'authors', 'ratings_count']].sort_values(by=['authors', 'ratings_count'], ascending=[True, False])
author_books = author_books.reset_index(drop=True)

author_books.head()

In [ ]:
author_rec_model = agg.merge(author_books, how='left', on=['authors'])

In [ ]:
author_rec_model.head()

In [ ]:
author_rec_model[author_rec_model['user_id'] == 30944]['book_id'].values

In [ ]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    author_rec_model_ = author_rec_model[author_rec_model['user_id'] == user]['book_id'].values
    for rec in author_rec_model_: 
        rec_list.append(rec)
    
    if len(rec_list) < 200:
        for i in popular_rec_model[0:200]:
            rec_list.append(rec)
        
    total_rec_list[user] = rec_list[0:200]

In [ ]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

### 후처리
- 내가 읽은 책은 추천해주면 안됨
- 내가 읽은 언어와 맞는 책을 추천해줘야함

In [ ]:
# 내가 읽은 책의 목록을 추출 
read_list = train.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
read_list.head()

In [ ]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    author_rec_model_ = author_rec_model[author_rec_model['user_id'] == user]['book_id'].values
    seen = read_list[read_list['user_id'] == user]['unique'].values[0]
    for rec in author_rec_model_: 
        if rec not in seen:
            rec_list.append(rec)
    
    if len(rec_list) < 200:
        for i in popular_rec_model[0:200]:
            if rec not in seen:
                rec_list.append(rec)

    total_rec_list[user] = rec_list[0:200]

In [ ]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

In [ ]:
# 내가 읽을 수 있는 언어의 목록을 추출 
## User에 대한 메타정보가 있으면 쉽게 추출가능하지만, 현재는 없으므로 직접 생성 
## Ratings에서 읽은 책들의 언어를 전부 수집해서 해당 언어의 책들을 가능한 언어로 설정 
language = pd.merge(train, books[['book_id', 'language_code']], how='left', on='book_id')

In [ ]:
language_list = language.groupby(['user_id'])['language_code'].agg({'unique'}).reset_index()
language_list.head()